# GUV Demo 3.2: IOT scenario - Partner/Producer - configure and create data

1. Imports
2. Define variables
3. Get access to universe
4. Configure universe
5. Create external workcenter
6. Create a relation between existing person and new workcenter
7. Create a power consumption activity for the workcenter

#### 1. Imports

In [ ]:
# import libraries 
import os
import random
import datetime as dt
from dotenv import load_dotenv
from guvcli import GUV


#### 2. Variables

In [ ]:
# prepare 
load_dotenv()

GUV_ENDPOINT        = os.getenv("GUV_ENDPOINT")
GUV_UNIVERSE        = os.getenv("GUV_UNIVERSE")

PERSON_TYPE         = "Person"
PERSON_KEY          = "0000000021"

WORKCENTER_TYPE     = "Workcenter"
WORKCENTER_TEXT     = "Arbeitsplatz"
WORKCENTER_COUNT    = 10

ACTIVITY_TYPE       = "WorkcenterPowerConsumption"
RELATION_TYPE       = "isWorkcenterResponsible"

#### 3. Get access to universe

In [ ]:
# prepare universe access
connector = GUV.connect(GUV_ENDPOINT, GUV_UNIVERSE)
if connector.is_valid():
    print("GUV connected: ", connector.info())
else:
    print("GUV connection failed")

#### 4. Configure universe
Create some new types if not exists.

In [ ]:
# create a new object type and create objects
tobj_wc = connector.object_types().create_object_type({
    "key": "Workcenter",
    "title": "Arbeitsplatz"
})

print(f"Created Workcenter Type: {tobj_wc}")

# create relation type
trel_wc_resp = connector.relation_types().create_relation_type({
    "key": RELATION_TYPE,
    "title": "Arbeitplatz - Verantwortlicher",
    "title_outbound": "ist verantwortlich für den Arbeitsplatz",
    "title_inbound": "hat Arbeitsplatzverantwortliche",
})
print(f"Created relation type: {trel_wc_resp}")

# create relation type 
tact_power = connector.activity_types().create_activity_type({
    "key": ACTIVITY_TYPE,
    "title": "Arbeitplatz - Stromverbrauch",
    "value_unit": "KWh"
})
print(f"Created activity type: {tact_power}")

#### 5. Create external workcenters

In [ ]:
workcenter_mgr = connector.objects(WORKCENTER_TYPE)
for index in range(10):
    workcenter_inst = workcenter_mgr.create_object({
      "key": f"wc{index}",
      "title": f"{WORKCENTER_TEXT} {index}"

    })
    print(f"Created Workcenter for index {index}: {workcenter_inst}")

#### 6. Create a relation between existing person and new workcenter

In [ ]:
# get a focused workcenter
wc_focus = workcenter_mgr.get_record_by_key("wc1")
print(f"Focused Workcenter: {wc_focus}")

# get a focused person
person_mgr = connector.objects(PERSON_TYPE)
person_focus = person_mgr.get_record_by_key(PERSON_KEY)
print(f"Focused Person: {person_focus}")

# create a new relation between person and focused workcenter
irel_rec = connector.relation_types().create_relation_for(
    type_key=RELATION_TYPE,
    source=person_focus,
    target=wc_focus,
    data={
        "key": f"{person_focus.get_value_key()}-{wc_focus.get_value_key()}",
        "title": f"created from python: {person_focus.get_value_title()} --> {wc_focus.get_value_title()}"
    }
)
print("IREL created: ", irel_rec)

#### 7. Create a power consumption activity

In [ ]:
# create a new power consumption for focused workcenter
used_power = random.randint(1,50)
time_to    = dt.datetime.now()
time_from  = time_to - dt.timedelta(minutes=30) 

iact_rec = connector.activity_types().create_activity_for(ACTIVITY_TYPE, wc_focus, {
        "key": f"python-demo-{time_to.isoformat()}",
        "title": f"Stromverbrauch {used_power} KWh",
        "value": used_power,
        "started_at": time_from.isoformat(),
        "finished_at": time_to.isoformat()
    })
print("IACT created: ", iact_rec)